In [ ]:
#Install tqdm to have progress bar
!pip install tqdm

#install necessary pkg to make connection with elasticsearch domain
!pip install requests
!pip install requests-aws4auth
!pip install "sagemaker>=2.0.0<3.0.0"

In [ ]:
# %%capture
# import IPython
# import sys

# !{sys.executable} -m pip install ipywidgets
# IPython.Application.instance().kernel.do_shutdown(True)  # has to restart kernel so changes are used

In [ ]:
import boto3
import re
import time
import os
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()

s3_resource = boto3.resource("s3")
s3 = boto3.client('s3')

print(f'SageMaker SDK Version: {sagemaker.__version__}')

In [ ]:
cfn = boto3.client('cloudformation')

def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to use for the rest of the demo
cloudformation_stack_name = "sbert"

outputs = get_cfn_outputs(cloudformation_stack_name)

backend_bucket = outputs['s3BucketBackend']
lambda_bucket = outputs['s3BucketLambda']

outputs

In [ ]:
!pip install install transformers[torch]

In [ ]:
#Save the model to disk which we will host at sagemaker
from transformers import AutoTokenizer, AutoModel
saved_model_dir = 'transformer'
os.makedirs(saved_model_dir, exist_ok=True)

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/distilbert-base-nli-stsb-mean-tokens")
model = AutoModel.from_pretrained("sentence-transformers/distilbert-base-nli-stsb-mean-tokens") 

tokenizer.save_pretrained(saved_model_dir)
model.save_pretrained(saved_model_dir)

In [ ]:
#Defining default bucket for SageMaker pretrained model hosting
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
#zip the model in tar.gz format
!cd transformer && tar czvf ../model.tar.gz *

In [ ]:
#Upload the model to S3

inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='sentence-transformers-model')
inputs

In [ ]:
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role

class StringPredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')
           

In [ ]:
pytorch_model = PyTorchModel(model_data = inputs, 
                             role=role, 
                             entry_point ='inference.py',
                             source_dir = './code',
                             py_version = 'py3', 
                             framework_version = '1.7.1',
                             predictor_cls=StringPredictor)

predictor = pytorch_model.deploy(instance_type='ml.t2.xlarge', 
                                 initial_instance_count=1, 
                                 endpoint_name = f'sbert-{int(time.time())}')


In [ ]:
# Doing a quick test to make sure model is generating the embeddings
import json
payload = 'a yellow dress that comes about to the knees'
features = predictor.predict(payload)
embedding = json.loads(features)

embedding

## Boto3 Method

In [ ]:
#calling SageMaker Endpoint
client = boto3.client('sagemaker-runtime')
ENDPOINT_NAME = predictor.endpoint
response = client.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                       ContentType='text/plain',
                                       Body=payload)

response_body = json.loads((response['Body'].read()))


## Deploying a full-stack NLU search application

In [ ]:
s3_resource.Object(backend_bucket, 'backend/template.yaml').upload_file('./backend/template.yaml', ExtraArgs={'ACL':'public-read'})
s3_resource.Object(lambda_bucket, 'lambda.zip').upload_file('./backend/lambda.zip', ExtraArgs={'ACL':'public-read'})


sam_template_url = f'https://{backend_bucket}.s3.amazonaws.com/backend/template.yaml'

# Generate the CloudFormation Quick Create Link

print("Click the URL below to create the backend API for sbert:\n")
print((
    'https://console.aws.amazon.com/cloudformation/home?region=eu-central-1#/stacks/create/review'
    f'?templateURL={sam_template_url}'
    '&stackName=sbert-api'
    f'&param_BucketName={outputs["s3BucketBackend"]}'
    f'&param_SagemakerEndpoint={predictor.endpoint}'
))

In [ ]:
# Review the content of the Lambda function code.
!pygmentize backend/lambda/app.py

In [ ]:
import json
api_endpoint = get_cfn_outputs('sbert-api')['TextSimilarityApi']

api_endpoint

## Cleanup

Make sure that you stop the notebook instance and delete the Amazon SageMaker endpoint.

In [ ]:
# Delete the endpoint
predictor.delete_endpoint()

# Empty S3 Contents
bucket_resource = s3_resource.Bucket(backend_bucket)
bucket_resource.objects.all().delete()